In [1]:

import cv2


In [2]:
import numpy as np
import os

In [3]:
import sys

sys.path.insert(0, r"/home/pi/teste/Project_NEURAL/Mask_RCNN-master/mrcnn")

import config
import utils

from model2 import MaskRCNN

from pathlib import Path

sys.path.append('/usr/local/lib/python3.7/dist-packages')
import nest_asyncio
nest_asyncio.apply()

Using TensorFlow backend.


In [4]:
class MaskRCNNConfig(config.Config):
    NAME = "coco_pretrained_model_config"
    IMAGES_PER_GPU = 1
    GPU_COUNT = 1
    NUM_CLASSES = 1 + 80  # COCO dataset has 80 classes + one background class
    DETECTION_MIN_CONFIDENCE = 0.6


In [5]:
def get_car_boxes(boxes, class_ids):
    car_boxes = []

    for i, box in enumerate(boxes):
        # If the detected object isn't a car / truck, skip it
        if class_ids[i] in [3, 8, 6]:
            car_boxes.append(box)

    return np.array(car_boxes)

In [6]:
# Root directory of the project
ROOT_DIR = Path("/home/pi/teste/Project_NEURAL")


In [7]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [8]:
# Create a Mask-RCNN model in inference mode
model = MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=MaskRCNNConfig())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
box_ind is deprecated, use box_indices instead


In [9]:
# Load pre-trained model
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [10]:
# Location of parking spaces
parked_car_boxes = None

In [ ]:
# Loop over each frame of video
while 1:
    video_capture = cv2.VideoCapture(0)
    ret, frame = video_capture.read()
    cv2.imwrite("bobo.jpg", frame)
    if not 9:
        break

    # Convert the image from BGR color (which OpenCV uses) to RGB color
    rgb_image = frame[:, :, ::-1]
    
    # Run the image through the Mask R-CNN model to get results.
    results = model.detect([rgb_image], verbose=0)
    # Mask R-CNN assumes we are running detection on multiple images.
    # We only passed in one image to detect, so only grab the first result.
    r = results[0]

    # The r variable will now have the results of detection:
    # - r['rois'] are the bounding box of each detected object
    # - r['class_ids'] are the class id (type) of each detected object
    # - r['scores'] are the confidence scores for each detection
    # - r['masks'] are the object masks for each detected object (which gives you the object outline)

    # Filter the results to only grab the car / truck bounding boxes
    car_boxes = get_car_boxes(r['rois'], r['class_ids'])

    print("Cars found in frame of video:")
    
    # FLag de presença de carros fora do lugar
    carrofora = 0
    A_1 = 0
    A_2 = 0
    A_3 = 0
    A_4 = 0
    
    # Draw each box on the frame
    for box in car_boxes:
        print("Carros: ", box)
        y1, x1, y2, x2 = box
        if (y1>60 and y1<130 and x1>=0 and x1<130 and y2>100 and y2<240 and x2>45 and x2<210):
            #print("Carro no lugar A-1")
            A_1 = 1

        if(y1>60 and y1<150 and x1>90 and x1<250 and y2>130 and y2<240 and x2>150 and x2<310):
            #print ("Carro no lugar A-2")
            A_2 = 1
    
        if(y1>60 and y1<160 and x1>220 and x1<355 and y2>135 and y2<240 and x2>275 and x2<425):
            #print ("Carro no lugar A-3")
            A_3 = 1
    
        if(y1>60 and y1<165 and x1>330 and x1<465 and y2>140 and y2<245 and x2>385 and x2<535):
            #print ("Carro no lugar A-4")
            A_4 = 1
            
        if (y2> 240):
            carrofora =1
            

            
        
        # Draw the box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 1)
        
        #Write in file compare.txt
        
    with open("compare.txt","w") as volatileFile:    
        volatileFile.write("A-1 = " + str(A_1) + "\nA-2 = " + str(A_2) + "\nA-3 = " + str(A_3) + "\nA-4 = " + str(A_4) + "\nFORA = " + str(carrofora) )
    
    
    %run "connectionAzure.py"
    # Show the frame of video on the screen
    cv2.imshow('Video', frame)
    
    video_capture.release()
    
    
    # Hit 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# Clean up everything when finished
cv2.destroyAllWindows()


Cars found in frame of video:
Carros:  [ 60   1 144  82]
Carros:  [ 68 125 148 191]
Carros:  [ 72 374 158 439]
Carros:  [ 72 263 158 322]
Sao Iguais!
Cars found in frame of video:
Carros:  [ 61   0 144  83]
Carros:  [ 71 124 151 190]
Carros:  [ 73 264 159 322]
Carros:  [ 75 374 159 440]
Sao Iguais!
Cars found in frame of video:
Carros:  [ 73 265 158 322]
0
Sending Message: {"A-1": "0", "A-2": "0", "A-3": "1", "A-4": "0", "FORA": "0"}
Cars found in frame of video:
Carros:  [ 74 264 159 322]
Sao Iguais!
Cars found in frame of video:
Carros:  [ 72 264 162 322]
Sao Iguais!
Cars found in frame of video:
Carros:  [182 379 246 527]
Carros:  [ 73 126 157 202]
Carros:  [ 72 264 162 322]
1
Sending Message: {"A-1": "1", "A-2": "1", "A-3": "1", "A-4": "0", "FORA": "1"}
Cars found in frame of video:
Carros:  [182 379 247 527]
Carros:  [ 75 265 159 322]
Carros:  [ 73 126 158 201]
Sao Iguais!
Cars found in frame of video:
Carros:  [181 382 247 527]
Carros:  [ 75 265 159 323]
Carros:  [ 74 127 158 20